## アプローチリストの作成

    ・インプット
      契約者情報
      直近webサイト登録者情報
      スコアリングデータ

    ・アウトプット
      スコア上位1000件のwebサイト未登録者リスト（顧客へのアプローチリストとしてに利用）

In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [2]:
files = glob('../data/*.csv')
files

['../data/df_main.csv', '../data/df_register.csv', '../data/df_scores.csv']

In [3]:
df_main = pd.read_csv(files[0])
df_main.shape

(15000, 3)

In [4]:
df_register = pd.read_csv(files[1])
df_register.shape

(2500, 1)

In [5]:
df_scores = pd.read_csv(files[2])
df_scores.shape

(5000, 2)

### 有効な契約者を絞り込む

In [6]:
valid_main = df_main[df_main['契約ステータスコード'] == '有効']
valid_main.shape

(12044, 3)

### Webサイトに登録していない契約者を取得

In [7]:
non_reg = valid_main[~valid_main['基本契約番号'].isin(df_register['基本契約番号'])]
non_reg.shape

(8000, 3)

### 有効契約者のスコアを付与

In [8]:
non_reg['基本契約番号'].nunique()

4562

In [9]:
non_reg_list = non_reg.drop_duplicates(subset=['基本契約番号'], keep='first')
approach_scores = pd.merge(non_reg_list, df_scores, on='基本契約番号', how='inner')
approach_scores.shape

(3076, 4)

In [10]:
approach_scores.head()

,基本契約番号,証券番号,契約ステータスコード,登録スコア
0,f97b25aba0,82a02f5005,有効,88
1,0c1ae83aa5,31d7def1ab,有効,82
2,83c27623f1,e03c3525e2,有効,84
3,70042afb50,318a4e98be,有効,21
4,45fdd9bf05,95b9281e7f,有効,96


### スコア順にソート

In [11]:
sorted_data = approach_scores.sort_values(by='登録スコア', ascending=False)
sorted_data.head()

,基本契約番号,証券番号,契約ステータスコード,登録スコア
37,6b78036108,b056f2fd7c,有効,97
2157,2af1f87a49,cf06f18edb,有効,97
2152,6ccf2256c0,6c37fa011e,有効,97
2400,8d01e4b18c,dd105256cd,有効,97
740,7c8a2e39d1,92d7a112e0,有効,97


### 上位1000件を取得（1000番目のスコアと同じスコアを持つ全ての契約者を含む）

In [12]:
top_1000_score = sorted_data['登録スコア'].iloc[999]
print('1000番目のスコア', top_1000_score)
top_1000 = sorted_data[sorted_data['登録スコア'] >= top_1000_score]
top_1000.shape

1000番目のスコア 65


(1037, 4)

### リストをCSVに出力

In [13]:
approach_list = top_1000[['基本契約番号', '登録スコア']]
approach_list.to_csv('../output/直近未登録者に対するアプローチリスト.csv', index=False)